In [2]:
%config ZMQInteractiveShell.ast_node_interactivity='all'

import numpy as np
import pandas as pd


# DataFrame列元素类型是dict时的操作

In [114]:
product_attri_data = {
    'product_id':[1, 1, 1, 2, 2],
    'attribute1':[{'colour':'red', 'size':(3,4,5)}, 
                {'colour2':'blue', 'size':(1, 5, 1)}, 
                {'colour3':'black', 'size':(0.25, 1, 3)}, 
                {'colour':'blue', 'size':(1, 1, 1)}, 
                {'colour1':'white', 'size':(2, 0.5, 1)}],
    'attribute2':[{'xx':'XXX'}, 
                {'yy':'YYY', 'yy2':'YYY2'}, 
                {'zz':'ZZZZ'}, 
                {}, 
                {}]
}

df = pd.DataFrame(product_attri_data)
df

# 合并两列的字典
def merge_attri(record):
#     print(type(record), '\n', 
#           type(record['attribute1']), record['attribute1'], '\n', 
#           type(record['attribute2']), record['attribute2'])
    record['attribute1'].update(record['attribute2'])  # 注意:这个方法返回None
    return record['attribute1']

df['attribute_merge'] = df.apply(merge_attri, 
                              axis=1, result_type='reduce')
df

# 同一列聚合后的字典合并
def merge_attri2(ser):
    from functools import reduce
    def dict_update(dic1, dic2):
        dic1.update(dic2)
        return dic1
    res = reduce(dict_update, ser.values)
    return res
df.groupby('product_id').agg({'attribute_merge': merge_attri2})

,product_id,attribute1,attribute2
0,1,"{'colour': 'red', 'size': (3, 4, 5)}",{'xx': 'XXX'}
1,1,"{'colour2': 'blue', 'size': (1, 5, 1)}","{'yy': 'YYY', 'yy2': 'YYY2'}"
2,1,"{'colour3': 'black', 'size': (0.25, 1, 3)}",{'zz': 'ZZZZ'}
3,2,"{'colour': 'blue', 'size': (1, 1, 1)}",{}
4,2,"{'colour1': 'white', 'size': (2, 0.5, 1)}",{}


<class 'pandas.core.series.Series'> 
 <class 'dict'> {'colour': 'red', 'size': (3, 4, 5)} 
 <class 'dict'> {'xx': 'XXX'}
<class 'pandas.core.series.Series'> 
 <class 'dict'> {'colour2': 'blue', 'size': (1, 5, 1)} 
 <class 'dict'> {'yy': 'YYY', 'yy2': 'YYY2'}
<class 'pandas.core.series.Series'> 
 <class 'dict'> {'colour3': 'black', 'size': (0.25, 1, 3)} 
 <class 'dict'> {'zz': 'ZZZZ'}
<class 'pandas.core.series.Series'> 
 <class 'dict'> {'colour': 'blue', 'size': (1, 1, 1)} 
 <class 'dict'> {}
<class 'pandas.core.series.Series'> 
 <class 'dict'> {'colour1': 'white', 'size': (2, 0.5, 1)} 
 <class 'dict'> {}


,product_id,attribute1,attribute2,attribute_merge
0,1,"{'colour': 'red', 'size': (3, 4, 5), 'xx': 'XXX'}",{'xx': 'XXX'},"{'colour': 'red', 'size': (3, 4, 5), 'xx': 'XXX'}"
1,1,"{'colour2': 'blue', 'size': (1, 5, 1), 'yy': '...","{'yy': 'YYY', 'yy2': 'YYY2'}","{'colour2': 'blue', 'size': (1, 5, 1), 'yy': '..."
2,1,"{'colour3': 'black', 'size': (0.25, 1, 3), 'zz...",{'zz': 'ZZZZ'},"{'colour3': 'black', 'size': (0.25, 1, 3), 'zz..."
3,2,"{'colour': 'blue', 'size': (1, 1, 1)}",{},"{'colour': 'blue', 'size': (1, 1, 1)}"
4,2,"{'colour1': 'white', 'size': (2, 0.5, 1)}",{},"{'colour1': 'white', 'size': (2, 0.5, 1)}"


,attribute_merge
product_id,
1,"{'colour': 'red', 'size': (0.25, 1, 3), 'xx': ..."
2,"{'colour': 'blue', 'size': (2, 0.5, 1), 'colou..."


In [59]:
data = {
    'product_id':[1, 1, 1, 2, 2],
    'attri_name':['colour', 'size', 'weight', 'colour', 'weight'],
    'attri_value':['red', '12x14', 15.6, 'blue', 10.2] 
}

df2 = pd.DataFrame(data)
df2

# 同一列聚合后的字典合并. (先将两列组合成元组,再由元组构成字典)
df2['attri_k_v'] = df2.apply(lambda row:(row['attri_name'], row['attri_value']), axis=1)
df2
df2['attri_merge'] = df2.groupby('product_id')\
    .agg({'attri_k_v': lambda attr_kv_ser:dict(attr_kv_ser.values)})

df2

,product_id,attri_name,attri_value
0,1,colour,red
1,1,size,12x14
2,1,weight,15.6
3,2,colour,blue
4,2,weight,10.2


,product_id,attri_name,attri_value,attri_k_v
0,1,colour,red,"(colour, red)"
1,1,size,12x14,"(size, 12x14)"
2,1,weight,15.6,"(weight, 15.6)"
3,2,colour,blue,"(colour, blue)"
4,2,weight,10.2,"(weight, 10.2)"


,product_id,attri_name,attri_value,attri_k_v,attri_merge
0,1,colour,red,"(colour, red)",NaN
1,1,size,12x14,"(size, 12x14)","{'colour': 'red', 'size': '12x14', 'weight': 1..."
2,1,weight,15.6,"(weight, 15.6)","{'colour': 'blue', 'weight': 10.2}"
3,2,colour,blue,"(colour, blue)",NaN
4,2,weight,10.2,"(weight, 10.2)",NaN


In [76]:
# 错误示例: 以下代码会产生 SettingWithCopyWarning -- 操作的是切片产生的副本.
# A value is trying to be set on a copy of a slice from a DataFrame.
# Try using .loc[row_indexer,col_indexer] = value instead
df3 = df2[['product_id', 'attri_name', 'attri_value']] # 这里产生的是“链接索引”
df3['new_col'] = df3[['attri_name', 'attri_value']].apply(
    lambda row:(row['attri_name'], row['attri_value']), 
    axis=1, result_type='reduce')
df3


df4 = df2.loc[:,['product_id', 'attri_name', 'attri_value']] # 这里产生的是副本,所以不会产生SettingWithCopyWarning
df4['new_col'] = df4[['attri_name', 'attri_value']]\
    .apply(lambda row:(row['attri_name'], row['attri_value']), 
           axis=1, result_type='reduce')
df4

# 直接操作切片,也不会产生SettingWithCopyWarning
df5 = df2[['attri_name', 'attri_value']]\
    .apply(lambda row:(row['attri_name'], row['attri_value']), 
           axis=1, result_type='reduce')
df5

E:\anaconda3\envs\py37_tf2x\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,product_id,attri_name,attri_value,new_col
0,1,colour,red,"(colour, red)"
1,1,size,12x14,"(size, 12x14)"
2,1,weight,15.6,"(weight, 15.6)"
3,2,colour,blue,"(colour, blue)"
4,2,weight,10.2,"(weight, 10.2)"


,product_id,attri_name,attri_value,new_col
0,1,colour,red,"(colour, red)"
1,1,size,12x14,"(size, 12x14)"
2,1,weight,15.6,"(weight, 15.6)"
3,2,colour,blue,"(colour, blue)"
4,2,weight,10.2,"(weight, 10.2)"


0     (colour, red)
1     (size, 12x14)
2    (weight, 15.6)
3    (colour, blue)
4    (weight, 10.2)
dtype: object

In [165]:
product_attri_data2 = {
    'product_id':[1, 1, 1, 2, 2],
    'attribute1':[{'colour':'red', 'size':(3,4,5)}, 
                {'colour2':'blue', 'size':(1, 5, 1)}, 
                {'colour3':'black', 'size':(0.25, 1, 3)}, 
                {'colour':'blue', 'size':(1, 1, 1)}, 
                {'colour1':'white', 'size':(2, 0.5, 1)}],
    
    # 有nan时,attribute1和attribute2两个字典无法正常合并
    'attribute2':[np.nan, 
                {'yy':'YYY', 'yy2':'YYY2'}, 
                {'zz':'ZZZZ'}, 
                np.nan, 
                {}]
    
#     没有nan时,attribute1和attribute2两个字典可正常合并
#     'attribute2':[{}, 
#                 {'yy':'YYY', 'yy2':'YYY2'}, 
#                 {'zz':'ZZZZ'}, 
#                 {}, 
#                 {}]
}

df5 = pd.DataFrame(product_attri_data2)
df5

def merge_attri_dict(row):
#     print(type(row), '\n',
#           type(row['attribute1']), row['attribute1'], '\n',
#           type(row['attribute2']), row['attribute2'], '\n')
    row['attribute1'].update(row['attribute2'])
    return row['attribute1']

# TypeError: 'float' object is not iterable
# 有nan时,直接合并两列字典会报错
# df5['attribute_merge'] = df5.apply(merge_attri_dict,
#     axis=1, result_type='reduce')


# OK
# 方法一
# 将'attribute2'列的nan替换成空字典{}, 现进行两列字典的合并
df5['attribute2'] = df5['attribute2'].apply(lambda item: item if item is not np.nan else {}) # 逐一修改Series元素类型
df5['attribute_merge'] = df5.apply(merge_attri_dict,
    axis=1, result_type='reduce')
df5

# 疑问: 怎么直接将一列中的nan替换成空字典
# df5['attribute2'].fillna({}) //不行


# 方法二
# 一开始不要存成dict格式,而是存成字符串结构,这样就可以使用使用b'{}'填充nan,最后再反序列化进行合并.
# 当列中的格式为json格式时,可以使用这种方法
# import json
# df5['attribute2'] = df5['attribute2'].apply(lambda item: json.dumps(item) if item is not np.nan else np.nan) # 将dict转换成json字符串(特意构造的)
# df5['attribute2'].fillna(b'{}', inplace=True)
# df5
# df5['attribute2'] = df5['attribute2'].apply(lambda item: json.loads(item))
# df5['attribute_merge'] = df5.apply(merge_attri_dict,
#     axis=1, result_type='reduce')
# df5


,product_id,attribute1,attribute2
0,1,"{'colour': 'red', 'size': (3, 4, 5)}",NaN
1,1,"{'colour2': 'blue', 'size': (1, 5, 1)}","{'yy': 'YYY', 'yy2': 'YYY2'}"
2,1,"{'colour3': 'black', 'size': (0.25, 1, 3)}",{'zz': 'ZZZZ'}
3,2,"{'colour': 'blue', 'size': (1, 1, 1)}",NaN
4,2,"{'colour1': 'white', 'size': (2, 0.5, 1)}",{}


,product_id,attribute1,attribute2,attribute_merge
0,1,"{'colour': 'red', 'size': (3, 4, 5)}",{},"{'colour': 'red', 'size': (3, 4, 5)}"
1,1,"{'colour2': 'blue', 'size': (1, 5, 1), 'yy': '...","{'yy': 'YYY', 'yy2': 'YYY2'}","{'colour2': 'blue', 'size': (1, 5, 1), 'yy': '..."
2,1,"{'colour3': 'black', 'size': (0.25, 1, 3), 'zz...",{'zz': 'ZZZZ'},"{'colour3': 'black', 'size': (0.25, 1, 3), 'zz..."
3,2,"{'colour': 'blue', 'size': (1, 1, 1)}",{},"{'colour': 'blue', 'size': (1, 1, 1)}"
4,2,"{'colour1': 'white', 'size': (2, 0.5, 1)}",{},"{'colour1': 'white', 'size': (2, 0.5, 1)}"
